In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

# read CSV dataset
dataset_source_url = "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv"
# csv_data = pd.read_csv(dataset_source_url, delimiter=";")

# # convert it to parquet and save to disk
# csv_data.to_parquet('~/OneDrive - Blue Altair/Desktop/Expts/MLFlow/winequality-white-brotli.parquet', 
#                     compression='brotli' # testing if file size and read speed is affected
#                     )

# read parquet file to perform operations on it
raw_data = pd.read_parquet('~/OneDrive - Blue Altair/Desktop/Expts/MLFlow/winequality-white-brotli.parquet').astype({
    'fixed acidity': 'float16',
    'volatile acidity': 'float16',
    'citric acid': 'float16',
    'residual sugar': 'float16',
    'chlorides': 'float16',
    'free sulfur dioxide': 'float16',
    'total sulfur dioxide': 'float16',
    'density': 'float16',
    'pH': 'float16',
    'sulphates': 'float16',
    'alcohol': 'float16',
    'quality': 'int8',
})

dataset_params = {
    'test_size': .1,
    'random_state': 42,
}

X_train, X_test, y_train, y_test = train_test_split(raw_data.loc[:, raw_data.columns != 'quality'], raw_data['quality'], 
                                                    test_size=dataset_params['test_size'],
                                                    random_state=dataset_params['random_state'], 
                                                    shuffle=True)

# balance the dataset
classes = np.unique(y_train)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weights = dict(zip(classes, weights))

In [15]:
import mlflow
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

train_pool = Pool(X_train, y_train)
test_pool = Pool(X_test, y_test)
training_params = {
    'learning_rate': .25,
    'depth': 8,
    'iterations': 10000,
    'class_weights': class_weights
}
model_name = f'dev.cb-wine-quality.ml'

# Create an instance of a PandasDataset
dataset = mlflow.data.from_pandas(
    raw_data, source=dataset_source_url, name="wine quality - white", targets="quality",
)

class MetricsCheckerCallback:
    def after_iteration(self, info):
        assert 'learn' in info.metrics
        assert len(info.metrics['learn']['MultiClass']) == info.iteration

        if info.iteration % 10 == 0: # do not store every iteration (reduce API calls)
            mlflow.log_metric('learn', info.metrics['learn']['MultiClass'][info.iteration - 1], step=info.iteration)
        
        return True


# train model
@mlflow.trace(name='train_catboost')
def train_model():
    bst = CatBoostClassifier(**training_params)
    bst.fit(train_pool, callbacks=[MetricsCheckerCallback()])
    mlflow.log_params(bst.get_all_params())
    return bst

# evaluate model and log metrics
@mlflow.trace(name='eval_catboost')
def eval_model(bst):
    preds = bst.predict(X_test)
    metrics = {
        'accuracy': accuracy_score(y_test.values, preds.flatten()),
        'f1': f1_score(y_test.values, preds.flatten(), average='weighted'),
        'recall': recall_score(y_test.values, preds.flatten(), average='weighted'),
        'precision': precision_score(y_test.values, preds.flatten(), average='weighted'),
    }
    mlflow.log_metrics(metrics)
    
    # Evaluate the logged model
    result = mlflow.evaluate(
        predictions='quality_pred',
        targets='quality',
        data=pd.concat([raw_data, pd.Series(bst.predict(raw_data).flatten(), name='quality_pred')], axis=1),
        model_type='classifier',
        evaluators=['default'],
    )
    print('Eval result', result)
    return metrics

c:\Users\yash.diniz\miniconda3\envs\mlflow\Lib\site-packages\mlflow\data\dataset_source_registry.py:149: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for 'https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv'. Exception: 
  return _dataset_source_registry.resolve(


In [16]:
import mlflow
from mlflow import MlflowClient
from mlflow.models import infer_signature

mlflow.set_tracking_uri('http://127.0.0.1:5000')
mlflow.set_experiment('Wine Quality')

# mlflow.set_tag('scale','minmax')
# X_train = (X_train - X_train.min())/(X_train.max()-X_train.min())
# X_test = (X_test - X_test.min())/(X_test.max()-X_test.min())

# mlflow.set_tag('scale','standardized')
# X_train = (X_train - X_train.mean())/(X_train.std())
# X_test = (X_test - X_test.mean())/(X_test.std())

signature = infer_signature(X_train, y_train)

# Log the Dataset to an MLflow run by using the `log_input` API
with mlflow.start_run() as run:
    client = MlflowClient()
    o_tags = client.get_latest_versions(model_name)[0].tags
    mlflow.log_input(dataset)
    mlflow.log_params(dataset_params)

    bst = train_model()

    # log model
    model_info = mlflow.catboost.log_model(
        cb_model=bst,
        artifact_path='wine_quality_white_model',
        signature=signature,
        registered_model_name=model_name,
    )

    metrics = eval_model(bst)

    # set model tags to track which model is better
    model = client.get_latest_versions(model_name)[0]
    client.set_model_version_tag(model_name, model.version, 'model_accuracy', metrics['accuracy'])
    client.set_model_version_tag(model_name, model.version, 'model_f1', metrics['f1'])
    client.set_model_version_tag(model_name, model.version, 'model_recall', metrics['recall'])
    client.set_model_version_tag(model_name, model.version, 'model_precision', metrics['precision'])

    # only set latest model as challenger if it has better metrics than older model
    if len(o_tags.keys()) > 0 and \
       float(o_tags['model_accuracy']) < metrics['accuracy'] and \
       float(o_tags['model_f1']) < metrics['f1'] and \
       float(o_tags['model_recall']) < metrics['recall'] and \
       float(o_tags['model_precision']) < metrics['precision']:
        client.set_registered_model_alias(model_name, 'challenger', model.version)
        # promote the challenger model to staging.
        mlflow.register_model(
            f'runs:/{run.info.run_id}/wine_quality_white_model',
            f'stag.cb-wine-quality.ml',
            tags=metrics,
        )

c:\Users\yash.diniz\miniconda3\envs\mlflow\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
C:\Users\yash.diniz\AppData\Local\Temp\ipykernel_21888\796523013.py:21: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future maj

0:	learn: 1.7113172	total: 57.2ms	remaining: 9m 31s
1:	learn: 1.6185433	total: 96.3ms	remaining: 8m 1s
2:	learn: 1.4976504	total: 135ms	remaining: 7m 29s
3:	learn: 1.3435417	total: 182ms	remaining: 7m 35s
4:	learn: 1.2649891	total: 232ms	remaining: 7m 42s
5:	learn: 1.1952735	total: 280ms	remaining: 7m 46s
6:	learn: 1.1228674	total: 325ms	remaining: 7m 44s
7:	learn: 1.0667115	total: 361ms	remaining: 7m 31s
8:	learn: 1.0048607	total: 398ms	remaining: 7m 21s
9:	learn: 0.9642171	total: 436ms	remaining: 7m 16s
10:	learn: 0.9211458	total: 492ms	remaining: 7m 26s
11:	learn: 0.8900438	total: 532ms	remaining: 7m 23s
12:	learn: 0.8748575	total: 570ms	remaining: 7m 17s
13:	learn: 0.8457545	total: 611ms	remaining: 7m 16s
14:	learn: 0.8230017	total: 656ms	remaining: 7m 16s
15:	learn: 0.7887983	total: 701ms	remaining: 7m 17s
16:	learn: 0.7647391	total: 738ms	remaining: 7m 13s
17:	learn: 0.7385163	total: 770ms	remaining: 7m 7s
18:	learn: 0.7231101	total: 801ms	remaining: 7m
19:	learn: 0.7094630	total

Registered model 'dev.cb-wine-quality.ml' already exists. Creating a new version of this model...
2025/03/26 12:32:13 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: dev.cb-wine-quality.ml, version 73
Created version '73' of model 'dev.cb-wine-quality.ml'.
c:\Users\yash.diniz\miniconda3\envs\mlflow\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\yash.diniz\miniconda3\envs\mlflow\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

Eval result <mlflow.models.evaluation.base.EvaluationResult object at 0x000001FC974B3830>


Registered model 'stag.cb-wine-quality.ml' already exists. Creating a new version of this model...
2025/03/26 12:32:15 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: stag.cb-wine-quality.ml, version 5
Created version '5' of model 'stag.cb-wine-quality.ml'.


🏃 View run bright-ox-837 at: http://127.0.0.1:5000/#/experiments/1/runs/d44d14356779403f9d0364aa70913ec9
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


[Trace(request_id=0831058f3e5a455cb50779749da3d251), Trace(request_id=81d731c7d635490ebd5e8f39e8e32b65)]

In [17]:
# Retrieve the run information
logged_run = mlflow.get_run(run.info.run_id)

# Retrieve the Dataset object
logged_dataset = logged_run.inputs.dataset_inputs[0].dataset

# View some of the recorded Dataset information
print(f"Dataset name: {logged_dataset.name}")
print(f"Dataset digest: {logged_dataset.digest}")
print(f"Dataset profile: {logged_dataset.profile}")
print(f"Dataset schema: {logged_dataset.schema}")

Dataset name: wine quality - white
Dataset digest: 7cbe3151
Dataset profile: {"num_rows": 4898, "num_elements": 58776}
Dataset schema: {"mlflow_colspec": [{"type": "float", "name": "fixed acidity", "required": true}, {"type": "float", "name": "volatile acidity", "required": true}, {"type": "float", "name": "citric acid", "required": true}, {"type": "float", "name": "residual sugar", "required": true}, {"type": "float", "name": "chlorides", "required": true}, {"type": "float", "name": "free sulfur dioxide", "required": true}, {"type": "float", "name": "total sulfur dioxide", "required": true}, {"type": "float", "name": "density", "required": true}, {"type": "float", "name": "pH", "required": true}, {"type": "float", "name": "sulphates", "required": true}, {"type": "float", "name": "alcohol", "required": true}, {"type": "integer", "name": "quality", "required": true}]}
